In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão Python:', python_version())

Versão Python: 3.9.7


## Redes Neurais Convolucionais

In [2]:
# Imports
import torch
import torch.nn as nn
import numpy as np
import torchvision
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [3]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Rodrigo Abreu" --iversions

Author: Rodrigo Abreu

numpy      : 1.19.5
torch      : 1.10.1
torchvision: 0.11.2



In [4]:
# Classe Para o Modelo
class ModeloCNN(nn.Module):
    
    # Construtor da Classe
    def __init__(self):
        super().__init__()
        
        # Camada de Entrada
        self.conv1 = nn.Conv2d(3, 16, 3, padding = 1)
        
        # Primeira Camada Oculta
        self.conv2 = nn.Conv2d(16, 32, 3, padding = 1)
        
        # Segunda Camada Oculta
        self.conv3 = nn.Conv2d(32, 64, 3, padding = 1)
        
        # Camada de Pooling
        self.pool = nn.MaxPool2d(2, 2)
        
        # Transformação Linear 1 (Camada Densa)
        self.linear1 = nn.Linear(64 * 4 * 4, 512)
        
        # Transformação Linear 2 (Camada Densa)
        self.linear2 = nn.Linear(512, 10) 
        
        # Camada de Dropout Para Regularização
        self.dropout = nn.Dropout(p = 0.3)
    
    # Método de Avanço da Rede
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = self.dropout(x)
        x = F.relu(self.linear1(x))
        x = self.dropout(x)
        x = self.linear2(x)
        return x

## Preparando as Transformações Para os Dados

In [5]:
# Transformações Para os Dados de Entrada
transformations = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
])

## Carregando e Dividindo os Dados em Treino, Teste e Validação

In [6]:
# Carrega os Dados de Treino
train_data = datasets.CIFAR10('CIFAR10', train = True, download = True, transform = transformations)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting CIFAR10\cifar-10-python.tar.gz to CIFAR10


In [7]:
# Carrega os Dados de Teste
test_data = datasets.CIFAR10('CIFAR10', train = False, download = True, transform = transformations)

Files already downloaded and verified


In [8]:
# Tamanho dos Datasets
len(train_data), len(test_data)

(50000, 10000)

In [9]:
# Tamanho Para Dados de Validação
validation_size = 0.2

In [10]:
# Tamanho Para Dados de Treino
training_size = len(train_data)

In [11]:
# Índice 
indices = list(range(training_size))
np.random.shuffle(indices)
index_split = int(np.floor(training_size * validation_size))

In [12]:
# Índices Para Dados de Treino e Validação
validation_indices, training_indices = indices[:index_split], indices[index_split:]

In [13]:
# Amostras de Treino e Validação
training_sample = SubsetRandomSampler(training_indices)
validation_sample = SubsetRandomSampler(validation_indices)

In [14]:
# Hiperparâmetros
batch_size = 16
n_epochs = 30
learning_rate = 0.01

In [15]:
# Gera as Amostras Finais de Dados
train_loader = DataLoader(train_data, batch_size = batch_size, sampler = training_sample)
valid_loader = DataLoader(train_data, batch_size = batch_size, sampler = validation_sample)
test_loader = DataLoader(train_data, batch_size = batch_size)

## Otimização do Modelo

In [16]:
# Cria o Modelo (Instância da Classe)
model = ModeloCNN()

In [17]:
model

ModeloCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=1024, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=10, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [18]:
# Identifica o Dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device.type

'cpu'

In [19]:
# Envia o Modelo para a GPU
model = model.to(device)

In [20]:
# Define a Função de Custo Baseada em Cross Entropia
criterion = nn.CrossEntropyLoss()

In [21]:
# Cria o Otimizador (Backward)
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

In [22]:
# Função Para Acurácia
def accuracy(preds, y):
    _, pred = torch.argmax(preds, 1)
    correct = pred.eq(target.data.view_as(pred))
    acc = correct.sum()/len(correct)
    return acc

## Treinamento

In [23]:
# Looop
for epoch in range(1, n_epochs+1):
    
    # Inicializa o Erro a Cada Epoch
    train_loss = 0.0
    valid_loss = 0.0
    
    # Cria Objeto de Treinamento do Modelo
    model.train()
    
    # Para Cada Batch de Dados Executa o Treinamento
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
        
    # Cria Objeto de Avaliação do Modelo    
    model.eval()
    
    # Para Cada Batch de Dados Executa a Avaliiação
    for batch_idx, (data, target) in enumerate(valid_loader):
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target)
        valid_loss += loss.item()*data.size(0)
        
    # Calcula o Erro na Epoch
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
    
    # Print
    print(f'| Epoch: {epoch:02} | Erro em Treino: {train_loss:.3f} | Erro em Validação: {valid_loss:.3f} |')

| Epoch: 01 | Erro em Treino: 2.037 | Erro em Validação: 1.734 |
| Epoch: 02 | Erro em Treino: 1.668 | Erro em Validação: 1.515 |
| Epoch: 03 | Erro em Treino: 1.508 | Erro em Validação: 1.403 |
| Epoch: 04 | Erro em Treino: 1.412 | Erro em Validação: 1.327 |
| Epoch: 05 | Erro em Treino: 1.343 | Erro em Validação: 1.268 |
| Epoch: 06 | Erro em Treino: 1.280 | Erro em Validação: 1.219 |
| Epoch: 07 | Erro em Treino: 1.228 | Erro em Validação: 1.166 |
| Epoch: 08 | Erro em Treino: 1.177 | Erro em Validação: 1.101 |
| Epoch: 09 | Erro em Treino: 1.144 | Erro em Validação: 1.076 |
| Epoch: 10 | Erro em Treino: 1.099 | Erro em Validação: 1.033 |
| Epoch: 11 | Erro em Treino: 1.065 | Erro em Validação: 1.000 |
| Epoch: 12 | Erro em Treino: 1.035 | Erro em Validação: 0.976 |
| Epoch: 13 | Erro em Treino: 1.007 | Erro em Validação: 0.967 |
| Epoch: 14 | Erro em Treino: 0.982 | Erro em Validação: 0.933 |
| Epoch: 15 | Erro em Treino: 0.959 | Erro em Validação: 0.915 |
| Epoch: 16 | Erro em Tre

In [25]:
# Salva o modelo treinado
torch.save(model.state_dict(), "modelos/cifar10.pth")

In [26]:
test_data

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: CIFAR10
    Split: Test
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-20.0, 20.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )